In [2]:
pdf.to_excel(r'D:\ODOC\jupyter_notebook\특허_크롤링\PATENT_DATA.xlsx', index=False)

In [3]:
import pandas as pd
import numpy as np
import re
pdf = pd.read_excel(r'D:\ODOC\jupyter_notebook\특허_크롤링\PATENT_DATA.xlsx')
pdf.head(3)

,APPLICATION_NUMBER,REGISTER_NUMBER,INGREDIENT_KOR,INGREDIENT_ENG,APPLICANT,PATENT_TEXT,PATENT_PURPOSE,DESCRIPTION,CATEGORY,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10,cat_11
0,10-2023-0188718,NaN,저온성 및 중온성 효소에 의한 식물 추출물의 제조 방법 및 그에 의한 항산화 및 항...,A Method for Producing a Vegetable Extract wit...,(주) 비에스티,저온성 및 중온성 효소에 의한 식물 추출물의 제조 방법 및 항산화 및 항갈변 특성의...,"항산화, 항갈변",본 발명은 저온성 및 중온성 효소에 의한 식물 추출물의 제조 방법 및 그에 의한 항...,"1,9",1,0,0,0,0,0,0,0,1,0,0
1,10-2023-0180301,NaN,용암해수 숙성 오미자 열매 추출물 등을 이용한 피부 주름 개선용 조성물 및 염증 반...,Composition for Improving Skin Wrinkles or Ant...,주식회사 두래,피부 주름 개선용 조성물 및 염증 반응 억제용 조성물,"콜라게나아제 생성 억제, 콜라겐 생성 촉진, 염증 반응 억제","본 발명은, 사람 각질형성세포인 HaCaT에서 미세먼지나 UV(자외선) 조사에 의해...","9,5",0,0,0,0,1,0,0,0,1,0,0
2,10-2023-0180860,NaN,앵두 추출물 및 수용성 콜라겐을 포함하는 화장료 조성물,Cosmetic composition comprising nanking cherry...,코스맥스 주식회사,살구앵두 추출물 및 콜라겐을 포함하는 화장료 조성물,"피부 관련 상태 예방, 개선 또는 치료","화장료 조성물에 관한 것으로, 상기 조성물은 살구앵두 추출물 및 콜라겐을 포함하고 ...","5,9",0,0,0,0,1,0,0,0,1,0,0


In [41]:
# 100개 직접 카테고리 지정해줌

for i in range(0,100):
    print(pdf['PATENT_PURPOSE'].iloc[i])
    print(pdf['요약'].iloc[i], end='\n\n')
    cat = input('입력: ')
    pdf['CATEGORY'].iloc[i] = cat

#### <성분분류>
-일부 카테고리 변경!!!!
1. 기미제거, 미백 (멜라닌, 색소침착)
2. 자외선 차단
3. 탈모방지
4. 헤어케어
5. 여드름 방지 및 제거, 항균, 염증억제, 알레르기
6. 청결 (클렌징)
7. 진정, 보습
8. 리페어(재생)
9. 주름개선, 피부탄력, 노화 방지 (콜라겐, 엘라스틴)
10. 각질제거(필링, 턴오버, 재생주기, 화장 들뜸)
11. 모공관리(블랙헤드, 화이트헤드, 면포, 모낭충)
12. 지성피부 (피지관리, 피지제거, 기름기)
13. 기타

#### (카테고리 변경 전 코드)

In [2]:
#카테고리별 핵심 키워드
keyword1 = ['천연','기미', '미백', '피부 미백', '멜라닌', '색소침착','효소', '황갈변', '피부 톤', '잡티', '주근깨','항갈변','투명','피부 색']
keyword2 = ['선스크린', '자외선','선크림','썬크림', 'UV','uv','차단용']
keyword3 = ['발모', '탈모', '모발','모발 성장','발모 촉진','탈모 방지','천연','두피','탈모 예방','탈모 치료제']
keyword4 = ['헤어 케어','모발 탄력','두피 착색','헤어스타일','샴푸','트리트먼트', '컨디셔닝','두피 개선','머릿결','두피 케어','염색','스타일링', '두피 각질','비듬','모발 영양']
keyword5 = ['여드름 방지', '항균', '항염', '염증 억제', '알레르기' ,'가려움', '항 가려움', '가려움증','염증 반응', '균제거', '염증', '세균성', '아토피','억제','완화','바이러스','항진균', '피부 염증', '염증 작용', '피부 질환','세균', '진균성','트러블']
keyword6 = ['청결','클렌징', '세정', '세안','세정성','세안성']
keyword7 = ['진정', '보습', '쿨링','보습용','수분 공급','촉촉','건조','피부 보습', '피부장벽 강화']
keyword8 = ['리페어','피부 상처', '재생', '치유', '치료', '피부장벽', '피부 상태', '손상']
keyword9 = ['주름 개선', '피부 탄력', '노화 방지', '콜라겐','항산화','리프팅', '천연','주름','피부 치밀도', '항노화']
keyword10 = ['각질 제거', '모공 수축', '모공 축소', '모공 개선', '모공 관리','모공 케어', '모공 수렴','피지', '노폐물','모공','여드름','부산물','오염 물질','배출']

keyword11 = ['발향','향균','심신','안정','제형']

In [9]:
#문서 전처리 -> 단어 토큰화, 1글자/조사/특수문자/영어 제거
from konlpy.tag import Okt
def extract_particles(text):
    okt = Okt()
    words = text.split()
    cleaned_words = []
    for word in words:
        morphemes = okt.pos(word)
        cleaned_word = [morpheme[0] for morpheme in morphemes 
                        if morpheme[1]!='Josa' 
                        and len(morpheme[0])>1
                        and re.match('[가-힣]+', morpheme[0])]
        cleaned_words.append(''.join(cleaned_word))
    return ' '.join(cleaned_words)

In [10]:
# 각 문서별로 적용 (제목+요약 텍스트 -> 전처리)
start = 0
end = len(pdf)

documents  = [a+'\n'+b for a, b in 
            zip(pdf.INGREDIENT_KOR.iloc[start:end], 
                pdf.DESCRIPTION.iloc[start:end])]
documents = [extract_particles(text) for text in documents]
# documents

In [12]:
idx = 190
print(pdf.INGREDIENT_KOR.iloc[start+idx])
print(pdf.DESCRIPTION.iloc[start+idx])
documents[idx]

변형된 손발톱 케어용 화장료 조성물 및 이의 제조방법
본 발명은 변형된 손발톱 케어용 화장료 조성물 및 이의 제조방법에 관한 것이다. 더욱 상세하게, 본 발명은 천연 소재를 이용하여, 변형된 손발톱에 의해 변형된 피부 및 변형되어 약해진 사이드윌의 재건에 효과적인 변형된 손발톱 케어용 화장료 조성물 및 이의 제조방법에 관한 것이다.


'변형 손발톱 케어 화장 조성  이의 제조방법  발명 변형 손발톱 케어 화장 조성  이의 방법   더욱 상세하게  발명 천연 소재 이용하여 변형 손발톱 의해 변형 피부  변형되어 약해진 사이드 재건 효과 변형 손발톱 케어 화장 조성  이의 방법  '

In [43]:
documents_save = documents
documents = [a+'\n'+b for a, b in 
            zip(pdf.PATENT_TEXT.iloc[start:end], 
                pdf.PATENT_PURPOSE.iloc[start:end])]

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

keywords = [keyword1, keyword2, keyword3, keyword4, keyword5,
            keyword6, keyword7, keyword8, keyword9, keyword10]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

similarity = []
for idx, keyword in enumerate(keywords):
    keyword_str = ' '.join(keyword)
    keyword_vector = vectorizer.transform([keyword_str])

    #keyword 카테고리에 대한, 문서들의 유사도
    sim = cosine_similarity(tfidf_matrix, keyword_vector)
    similarity.append(sim)

#각 문서에 해당하는 cat_칼럼 1로 수정
pdf.loc[start:end, 'cat_1':'cat_10'] = 0
for num, sim_values in enumerate(similarity):
    similar_indices = [i+start for i, sim in 
                    enumerate(sim_values) if sim >= 0.05]
    for low in similar_indices:
        pdf.loc[low, 'cat_'+str(num+1)] = 1   #num은 0부터 시작함

In [15]:
# 카테고리별 해당하는 문서 인덱스 확인
similar_documents = []
for num, sim_values in enumerate(similarity):
    similar_indices = [i+start for i, sim in 
                    enumerate(sim_values) if sim >= 0.05]
    similar_documents.append(similar_indices)

for i in similar_documents:
    print(i, '\n')

In [212]:
# for doc_index in range(len(similarity[0])):
#     print(f"문서 {start+doc_index}에 대한 카테고리별 유사도:")
    
#     for idx, keyword in enumerate(keywords):
#         sim_value = similarity[idx][doc_index]
#         print(f"  카테고리 {idx + 1}: {sim_value}")
    
#     print("\n")

In [45]:
document1 = list(pdf.DESCRIPTION[pdf.cat_1==1])
document2 = list(pdf.DESCRIPTION[pdf.cat_2==1])
document3 = list(pdf.DESCRIPTION[pdf.cat_3==1])
document4 = list(pdf.DESCRIPTION[pdf.cat_4==1])
document5 = list(pdf.DESCRIPTION[pdf.cat_5==1])
document6 = list(pdf.DESCRIPTION[pdf.cat_6==1])
document7 = list(pdf.DESCRIPTION[pdf.cat_7==1])
document8 = list(pdf.DESCRIPTION[pdf.cat_8==1])
document9 = list(pdf.DESCRIPTION[pdf.cat_9==1])
document10 = list(pdf.DESCRIPTION[pdf.cat_10==1])
docs = [document1,document2,document3,document4,document5,document6,
       document7,document8,document9,document10]

all_documents = list(pdf.DESCRIPTION.iloc[:end])

In [64]:
idx = 5657
print(pdf.INGREDIENT_KOR.iloc[start+idx])
print(pdf.DESCRIPTION.iloc[start+idx],'\n')
print(documents[idx])

for i in range(10):
    print(i+1,':',similarity[i][idx], end=' ')
    if similarity[i][idx]>0.05:
        print('★')
    else: print()

적어도 2종의 (폴리)글리세롤의 지방산 에스테르를 포함하는 조성물, 및 이의 화장품에서의 용도
본 발명은 나노에멀젼 또는 마이크로에멀젼 형태의 조성물로서,  - 12개 내지 20개의 탄소 원자를 함유하는 알킬 또는 알케닐 사슬을 포함하는 적어도 하나의 산 및 3개 내지 6개의 글리세롤 단위로부터 형성되는 폴리글리세롤의 지방산 에스테르로부터 선택되는 제1 폴리글리세롤의 지방산 에스테르;  - 6개 내지 18개의 탄소 원자를 함유하는 알킬 또는 알케닐 사슬을 포함하는 적어도 하나의 산 및 1개 내지 3개의 글리세롤 단위로부터 형성되는 (폴리)글리세롤의 지방산 에스테르로부터 선택되는 제2 (폴리)글리세롤의 지방산 에스테르 (c);  - 적어도 하나의 오르가노폴리실록산 탄성중합체;  - 서로 상이한 적어도 2종의 아크릴산계 중합체;  - 적어도 하나의 2-아크릴아미도메틸프로판설폰산(AMPS) 단위를 포함하는 적어도 하나의 양친매성 중합체;  - 적어도 하나의 오일;  - 물을 포함하는 조성물에 관한 것이다.  본 발명은 또한 이러한 조성물의 도포에 의한, 피부와 같은 케라틴 물질에 대한 미용 처치 방법에 관한 것이다. 

탄소 원자를 포함하는 지방산 에스테르를 사용한 에멀젼 조성물
피부 미용 처치
1 : [0.0278461] 
2 : [0.] 
3 : [0.] 
4 : [0.] 
5 : [0.01228633] 
6 : [0.] 
7 : [0.01192765] 
8 : [0.0311274] 
9 : [0.02403429] 
10 : [0.] 


In [7]:
for idx in pdf[pdf.cat_1==1].index:
    print(pdf.loc[idx, ['cat_1', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9', 'cat_10']])
    print(pdf.DESCRIPTION[idx], end='\n\n')

# pdf[pdf.cat_1==1].DESCRIPTION

In [166]:
document1 = list(pdf.DESCRIPTION[(pdf.cat_1==1)&(~pdf.CATEGORY.isna())])
document2 = list(pdf.DESCRIPTION[(pdf.cat_2==1)&(~pdf.CATEGORY.isna())])
document3 = list(pdf.DESCRIPTION[(pdf.cat_3==1)&(~pdf.CATEGORY.isna())])
document4 = list(pdf.DESCRIPTION[(pdf.cat_4==1)&(~pdf.CATEGORY.isna())])
document5 = list(pdf.DESCRIPTION[(pdf.cat_5==1)&(~pdf.CATEGORY.isna())])
document6 = list(pdf.DESCRIPTION[(pdf.cat_6==1)&(~pdf.CATEGORY.isna())])
document7 = list(pdf.DESCRIPTION[(pdf.cat_7==1)&(~pdf.CATEGORY.isna())])
document8 = list(pdf.DESCRIPTION[(pdf.cat_8==1)&(~pdf.CATEGORY.isna())])
document9 = list(pdf.DESCRIPTION[(pdf.cat_9==1)&(~pdf.CATEGORY.isna())])
document10 = list(pdf.DESCRIPTION[(pdf.cat_10==1)&(~pdf.CATEGORY.isna())])

docs = [document1,document2,document3,document4,document5,document6,
       document7,document8,document9,document10]

In [163]:
#조사제거/ 1글자 단어 제거/ 토큰화+명사만 추출/ 불용어 제거
from konlpy.tag import Okt

def extract_nouns(text):
    okt = Okt()
    nouns = okt.nouns(text)
    return nouns

def preprocess_text(text):
    nouns = extract_nouns(text)
    nouns = [w for w in nouns if (len(w)>=2) & (w not in not_words)]
    result = ' '.join(nouns)
    return result

not_words = ['발명', '추출', '효과', '기능', '성분', '효능', '방법', '함유', '중량', '내지', '제공',
    '안료', '가지', '부가', '상기', '코어', '여성', '유효', '식품', '형성', '활용', '미세',
    '배양', '신규', '유래', '예방', '사용', '환경', '배양', '이의', '분비', '사과', '함량',
    '화장품', '의약품', '하나', '이상', '능력', '물질', '수돗물', '분말', '상태','화장']

word_docs = []
for doc in docs:
    preprocessed_documents = []
    for document in doc:
        preprocessed_document = preprocess_text(document)
        preprocessed_documents.append(preprocessed_document)
    word_docs.append(preprocessed_documents)

In [165]:
# 각 문서집단별 대표단어 20개
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

all_documents = list(pdf.DESCRIPTION[~pdf.CATEGORY.isna()])

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_all = tfidf_vectorizer.fit_transform(all_documents)

top_words = []
num=0
for d in word_docs:
    num+=1
    tfidf_matrix = tfidf_vectorizer.transform(d)
    feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
    word_indices = tfidf_matrix.sum(axis=0).argsort()[0, ::-1][:10]
    word = feature_names[word_indices]
    print(f'Category{num}.',word[0][:10])

Category1. ['피부' '미백' '조성' '개선' '주름' '이용' '멜라닌' '항산화' '세포' '억제']
Category2. ['차단' '자외선' '조성' '피부' '흡수' '개선' '유기' '반응' '이용' '방지']
Category3. ['탈모' '모발' '촉진' '두피' '방지' '조성' '개선' '발모' '이용' '치료']
Category4. ['모발' '두피' '조성' '헤어' '개선' '탈모' '피부' '촉진' '베이스' '케어']
Category5. ['피부' '조성' '염증' '개선' '이용' '항균' '발효' '치료' '억제' '용도']
Category6. ['클렌징' '피부' '조성' '정제' '세안' '제거' '자극' '계면활성제' '비누' '오일']
Category7. ['피부' '조성' '보습' '개선' '발효' '이용' '장벽' '진정' '수분' '강화']
Category8. ['피부' '개선' '조성' '재생' '염증' '장벽' '세포' '발효' '용도' '주름']
Category9. ['피부' '개선' '조성' '주름' '이용' '항산화' '발효' '미백' '탄력' '염증']
Category10. ['피부' '피지' '모공' '조성' '개선' '여드름' '제거' '각질' '억제' '이용']


In [160]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

idx = pdf[pdf.CATEGORY.isna()].index
query_document = pdf.DESCRIPTION[idx[1000]]

similarities=[]
for doc in docs:
    #tfidf변환기에 문서집단 학습
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(doc)

    #문서집단-특정문서 간 유사도 계산
    query_tfidf_vector = vectorizer.transform([query_document])
    similarities.append(cosine_similarity(query_tfidf_vector, tfidf_matrix).mean())

In [161]:
for i in range(10):
    print(f'카테고리{i+1}.', similarities[i].round(4))
np.array(similarities).mean().round(4)

카테고리1. 0.0419
카테고리2. 0.0581
카테고리3. 0.0332
카테고리4. 0.0398
카테고리5. 0.0272
카테고리6. 0.0584
카테고리7. 0.0259
카테고리8. 0.031
카테고리9. 0.0278
카테고리10. 0.0492


0.0392

In [162]:
query_document

'본 발명의 피부 미백용 조성물은 황칠나무 캘러스 추출물을 바실러스 속(Bacillus sp.) JD3-7로 생물전환한 생물전환산물을 유효성분으로 함유한다. 또한, 본 발명의 황칠나무 캘러스 추출물의 생물전환 방법은 황칠나무 캘러스 추출물과 바실러스 속 JD3-7의 균체를 혼합하여 생물전환 반응용 조성물을 생성하는 단계; 및 상기 생물전환 반응용 조성물을 항온처리하는 단계;를 포함한다.'

In [289]:
pdf.to_excel(r'D:\ODOC\jupyter_notebook\특허_크롤링\PATENT_DATA.xlsx', index=False)

In [9]:
keyword = "모공"
index = pdf[pdf['DESCRIPTION'].str.contains(keyword)
          &(pdf['CATEGORY'].isna())].index
idx = index[:10]

for i in idx:
    print(i,':', pdf.DESCRIPTION[i])
    cat = input('카테고리 입력:')
    pdf.CATEGORY[i] = str(cat)
    print()

In [47]:
# # 카테고리별 칼럼 만들기
for cat in range(1, 12):
    pdf[f'cat_{cat}'] = 0

In [153]:
#원-핫 인코딩
import re
category_onehot = pdf[~pdf.CATEGORY.isna()]

for idx, row in category_onehot.iterrows():
    row['CATEGORY'] = re.sub(r'\.', ',', row['CATEGORY'])
    category_list=[]
    for x in row['CATEGORY'].split(','):
        match = re.search(r'\d+',x)
        if match: category_list.append(match.group())
    
    for cat in category_list:
        pdf.at[idx, f'cat_{cat}'] = 1

In [8]:
# pdf.iloc[0:100]
idx = pdf[pdf.cat_8==1].index

for i in idx:
    print(i)
    print(pdf['PATENT_PURPOSE'].iloc[i], pdf['CATEGORY'].iloc[i])
    print(pdf['요약'].iloc[i], end='\n\n')